In [1]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [2]:
gameid = 2021020180
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [3]:
time.sleep(27000)
# Use for delay start
pullurl = starturl + str(gameid) + endurl
pullurl

'https://statsapi.web.nhl.com/api/v1/game/2021020180/feed/live.json'

In [4]:
while gameid <= 2021020317:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings') and str(game_date) == str(tomorrow):
        #print(x)
        #x=x+1
        finalll == 0
        while finalll != 1:
            pullurl = starturl + str(gameid) + endurl
            try:
                with urllib.request.urlopen(pullurl) as url:
                    obj = json.loads(url.read().decode())
            except:
                print(f'The URL is broke, bruh. Try again.')
            allPlays = obj["liveData"]["plays"]['allPlays']
            if period < 3:
                time.sleep(5)
            hometeamscore = 0
            awayteamscore = 0
            for play in allPlays:
                period = play['about']['period']
                event = play['result']['event']
                timeee = play['about']['periodTime']
                description = play['result']['description']
                if event == 'Goal':
                    if str(home_team) == 'Los Angeles Kings':
                        if play['team']['name'] == 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                    if str(home_team) != 'Los Angeles Kings':
                        if play['team']['name'] != 'Los Angeles Kings':
                            hometeamscore += 1
                        else:
                            awayteamscore += 1
                if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                    gamedetail.append(f'Period: {period}-{timeee}, {event}')
                    if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                        print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                        #Period Ready
                        #Giveaway
                        #Takeaway
                        #Hit
                    #elif event == 'Game Scheduled':
                        #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                        #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                        #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                        #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                        #time.sleep(8)
                    elif event == 'Period Start':
                        try:
                            api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                        #Period Start
                    elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                        try:
                            api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                            print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        time.sleep(8)
                        #Faceoff
                        #Blocked Shot
                        #Shot
                        #Penalty
                        #Missed Shot
                        #Stoppage
                        #Period Official
                        #Game End
                        #Game Official
                    elif event == 'Goal':
                        try:
                            api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                            print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        time.sleep(8)
                        #Goal
                    elif event == 'Period End' and (period == 1 or period == 2):
                        try:
                            api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(1080)
                        #Period End
                    elif event == 'Period End' and (period == 3):
                        try:
                            api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    elif event == 'Period End' and (period == 4):
                        try:
                            api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                            print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        except:
                            print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        time.sleep(8)
                    else:
                        time.sleep(0)
            print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020180-Final-2021-11-09-2021-11-12 -- New York Rangers vs Florida Panthers
2021020181-Final-2021-11-09-2021-11-12 -- Washington Capitals vs Buffalo Sabres
2021020182-Final-2021-11-10-2021-11-12 -- Boston Bruins vs Ottawa Senators
2021020183-Final-2021-11-10-2021-11-12 -- Montréal Canadiens vs Los Angeles Kings
2021020184-Final-2021-11-10-2021-11-12 -- Detroit Red Wings vs Edmonton Oilers
2021020185-Final-2021-11-10-2021-11-12 -- Tampa Bay Lightning vs Carolina Hurricanes
2021020186-Final-2021-11-10-2021-11-12 -- New Jersey Devils vs Florida Panthers
2021020187-Final-2021-11-10-2021-11-12 -- Chicago Blackhawks vs Pittsburgh Penguins
2021020188-Final-2021-11-10-2021-11-12 -- Winnipeg Jets vs St. Louis Blues
2021020189-Final-2021-11-10-2021-11-12 -- Calgary Flames vs San Jose Sharks
2021020190-Final-2021-11-10-2021-11-12 -- Vancouver Canucks vs Anaheim Ducks
2021020191-Final-2021-11-10-2021-11-12 -- Vegas Golden Knights vs Seattle Kraken
2021020192-Final-2021-11-11-2021-11-12 -- Phila

Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 1-02:32, Blocked Shot-Alexander Edler shot blocked shot by Michael Del Zotto #GoKingsGo
NOT TWEETED---Period: 1-02:45, Hit-Parker Kelly hit Alexander Edler
Period: 1-02:50, Stoppage-Puck in Netting #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 1-02:50, Faceoff-Phillip Danault faceoff won against Nick Paul #GoKingsGo
NOT TWEETED---Period: 1-02:54, Hit-Drake Batherson hit Matt Roy
NOT TWEETED---Period: 1-03:08, Hit-Drake Batherson hit Matt Roy
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Ange

Period: 1-12:09, Stoppage-Puck in Benches #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 1-12:09, Faceoff-Nick Paul faceoff won against Phillip Danault #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa 

Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period Starting: 2 --- Score: Ottawa Senators-0, Los Angeles Kings-0 #GoKingsGo
Period: 2-00:00, Faceoff-Anze Kopitar faceoff won against Josh Norris #GoKingsGo
NOT TWEETED---Period: 2-00:21, Giveaway-Giveaway by Thomas Chabot
Period: 2-00:25, Blocked Shot-Tobias Bjornfot shot blocked shot by Brady Tkachuk #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 2-00:44, Stoppage-Icing #GoKingsGo
Period: 2-00:44, Faceoff-Anze Kopitar faceoff won against Nick Paul #GoKingsGo
Period: 2-00:54, Stoppage-Icing #GoKingsGo
Ottawa Se

Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 2-08:47, Faceoff-Josh Norris faceoff won against Rasmus Kupari #GoKingsGo
Period: 2-09:59, Blocked Shot-Chris Tierney shot blocked shot by Mikey Anderson #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Period: 2-10:11, Shot-Dustin Brown Wrist Shot saved by Filip Gustavsson #GoKingsGo
Period: 2-10:12, Stoppage-Goalie Stopped #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angeles Kings-0
Ottawa Senators-0, Los Angele

...

Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Period: 3-10:59, Faceoff-Brady Tkachuk faceoff won against Anze Kopitar #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, L

Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Period: 3-15:31, Stoppage-Puck in Netting #GoKingsGo
Period: 3-15:31, Blocked Shot-Dustin Brown shot blocked shot by Thomas Chabot #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1


Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
Ottawa Senators-0, Los Angeles Kings-1
NOT TWEETED---Period: 3-16:36, Giveaway-Giveaway by Thomas Chabot
NOT TWEETED---Period: 3-16:42, Takeaway-Takeaway by Andreas Athanasiou
Period: 3-16:50, Score: Ottawa Senators-0 ~ Los Angeles Kings-2, Goal-Andreas Athanasiou (2) Tip-In, assists: Alexander Edler (4), Phillip Danault (6), #GoKingsGo
Ottawa Senators-0, Los Angeles Kings-

Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
NOT TWEETED---Period: 3-19:51, Hit-Brady Tkachuk hit Alexander Edler
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kings-2
Ottawa Senators-0, Los Angeles Kin

2021020256-Preview-2021-11-19-2021-11-12 -- Arizona Coyotes vs Columbus Blue Jackets
2021020257-Preview-2021-11-19-2021-11-12 -- Vegas Golden Knights vs Detroit Red Wings
2021020258-Preview-2021-11-19-2021-11-12 -- Anaheim Ducks vs Carolina Hurricanes
2021020259-Preview-2021-11-20-2021-11-12 -- Vancouver Canucks vs Winnipeg Jets
2021020260-Preview-2021-11-20-2021-11-12 -- Seattle Kraken vs Colorado Avalanche
2021020261-Preview-2021-11-21-2021-11-12 -- Toronto Maple Leafs vs Pittsburgh Penguins
2021020262-Preview-2021-11-21-2021-11-12 -- Montréal Canadiens vs Nashville Predators
2021020263-Preview-2021-11-20-2021-11-12 -- Ottawa Senators vs New York Rangers
2021020264-Preview-2021-11-20-2021-11-12 -- Tampa Bay Lightning vs New Jersey Devils
2021020265-Preview-2021-11-20-2021-11-12 -- Florida Panthers vs Minnesota Wild
2021020266-Preview-2021-11-21-2021-11-12 -- New York Islanders vs Calgary Flames
2021020267-Preview-2021-11-21-2021-11-12 -- Philadelphia Flyers vs Boston Bruins
202102026

In [5]:
#use to catch up to live
for play in allPlays:
    if f'Period: {period}-{timeee}, {event}' not in gamedetail:
        gamedetail.append(f'Period: {period}-{timeee}, {event}')
        print(play)